In [1]:
from bedarev_semantic.base import *
from bedarev_semantic.compile import SemanticVocabulary


information = [
    #
    Synonyms(
        'ДолжностноеЛицо',
        {'рабочий', 'сотрудник', 'государственный служащий'}
    ),
    Synonyms(
        'РФ',
        {'рф', 'россия', 'российский федерация'}
    ),
    Synonyms(
        'Семья',
        {'семья', 'ячейка общество'}
    ),
    Synonyms(
        'ЦБ',
        {'цб', 'центральный банк', 'отечественный цб'}
    ),
    Synonyms(
        'ВВП',
        {'ввп', 'валовый продукт'}
    ),
    Synonyms(
        'ОбразованныйЧеловек',
        {'учёный', 'эксперт', 'академик'}
    ),
    GeneralPrivate(
        'Государство',
        {
            Synonyms('РФ'),
            'cccp',
            'китай',
            'сша'
        },
        values={'государство', 'страна', 'федерация'}
    ),
    GeneralPrivate(
        'ДенежныеСредства',
        {
            'рубль',
            'доллар',
        },
        {'деньга', 'средства', 'криптовалюта', 'валюта'}
    ),
    #
    GeneralPrivate(
        'Числительные',
        {
            'нуль',
            'три',
            'два',
            'миллион',
            'миллиардмиллиард',
        },
        {'число', 'количество'}
    ),
    # транзакции -> приход и расход
    GeneralPrivate(
        'Субъект',
        {
            'округ',
            'страна',
        },
        {'субъект', }
    ),
    PartWhole(
        'ПланетнаяСистема',
        {
            GeneralPrivate('Звезда', {'солнце'}, {'звезда'}),
            GeneralPrivate('Спутник', {'луна'}, {'спутник'}),
            GeneralPrivate('Планета', {'земля'}, {'планета'})
        },
        {'планетный система', }
    ),
    GeneralPrivate(
        'НаучнаяОбласть',
        {
            'математика',
            'физика',
            'информационные технологии',
        },
        {'наука'}
    ),
    Synonyms(
        'ПротопланетныйДиск',
        {'протопланетный диск', 'протопланетный кольцо'}
    ),
    Synonyms(
        'Затрата',
        {'затрата', 'издержка'}
    ),
    Synonyms(
        'Финансирование',
        {'кредит', 'кредитование',
         'вклад', 'финансирование', 'инвестиция'}
    ),
    Synonyms(
        'Налогообложение',
        {'налог', 'налоговый нагрузка'}
    ),
    Synonyms(
        'Регулирование',
        {'регулирование', 'управление'}
    ),
    Synonyms(
        'Банк',
        {'банк', 'банковский система'}
    ),
    PartWhole(
        'Правительство',
        {
            'мгд',
            'институт'
        },
        {'правительство', 'власть'}
    ),
    GeneralPrivate(
        'ПолезноеИскопаемое',
        {
            'газ',
            'нефть',
        },
        {'полезный ископаемое', 'ресурс'}
    ),
    PartWhole(
        'ВременнойОтрезок',
        {
            'x годах',
            Synonyms(
                'НастоящееВремя',
                {'последний время', 'сегодня', 'сейчас', 'настоящее время'}
            ),
        },
    ),
    Synonyms(
        'Аглоритм',
        {'алгоритм', 'технология', 'схема', 'подход'}
    ),
]

voc = SemanticVocabulary(information)

In [2]:
from typing import Set, Tuple, Union

def get_values(obj: Union[str, SemanticObject]) -> Set[str]:
    if isinstance(obj, str):
        return {obj}
    all_values = set()
    for value in obj.values():
        all_values |= get_values(value)
    return all_values


def get_values_followers(obj: Union[SemanticObject]) -> Set[str]:
    all_values = set()
    for follower in obj.followers:
        if isinstance(obj, str):
            all_values.add(obj)
        else:
            all_values |= get_values(follower)
    return all_values


def top_down_algorithm(phrase: str, semantic_vocabulary: SemanticVocabulary,
                       weight_synonyms: float = 0.9,
                       weight_general_private: float = 0.5,
                       weight_part_whole: float = 0.25) -> Set[Tuple[float, str]]:
    result = set()
    
    for search in semantic_vocabulary.search(phrase):
        if isinstance(search, Synonyms):
            result |= {(weight_synonyms, v) for v in get_values(search) if v != phrase}
        elif isinstance(search, GeneralPrivate):
            result |= {(weight_synonyms, v) for v in get_values(search) if v != phrase}
            result |= {(weight_general_private, v) for v in get_values_followers(search)}
        elif isinstance(search, PartWhole):
            result |= {(weight_synonyms, v) for v in get_values(search) if v != phrase}
            result |= {(weight_part_whole, v) for v in get_values_followers(search)}
    
    return {(res[0], '_'.join(res[1].split(' '))) for res in result}

In [3]:
# print(top_down_algorithm('рф', voc))
# print(top_down_algorithm('государство', voc))

In [4]:
import re
import math
from collections import Counter
from typing import List, Tuple, Set, Dict

from pymorphy2 import MorphAnalyzer

DELIMITERS = (',', '.', '!', '?', ':', ';')

morph = MorphAnalyzer()

def normalize_line(line: str,) -> List[List[Set[str]]]:
    for delimiter in DELIMITERS:
            line = line.replace(delimiter, '\n')
    phrases = line.split('\n')
    
    normalize_sequence = []
    after_start = 0
    for phrase in phrases:
        normalize_phrase = []
        while phrase:
            search = re.search(r"('?[а-яА-ЯёЁ][а-яА-ЯёЁ]*(?:-[а-яА-ЯёЁ]+)*'?)", phrase)
            if not search:
                after_start += len(phrase)
                break
            else:
                start = search.start()
                end = search.end()
                phrase = phrase[end:]
                word = search.group(0).lower()
                forms = set([parse.normal_form for parse in morph.parse(word) if parse.normal_form != ' '])
                if forms:
                    normalize_phrase.append(((start+after_start, end+after_start), forms))
                after_start += end
        after_start += 1
        if normalize_phrase:
            normalize_sequence.append(normalize_phrase)

    return normalize_sequence

In [5]:
with open("./data/clear_text.txt", 'r', encoding='utf8') as fp:
    text = fp.read()

In [6]:
text_lines = text.split('\n')

In [7]:
normalize_lines = [normalize_line(line) for line in text_lines]

In [8]:
import pandas as pd


all_n_gramms = pd.read_csv('./data/all_n_gramms.csv')
correct_n_gram = all_n_gramms[(all_n_gramms.idf > 0.5) & ((all_n_gramms.frequency > 2) & (all_n_gramms.N < 5))]

MAX_N = correct_n_gram.N.max()
MAX_N

4

In [9]:
ALL_FORMS = set(form.replace(' ', '_') for form in correct_n_gram.form)

In [10]:
'полезный_ископаемое' in ALL_FORMS

True

In [11]:
import itertools

def get_n_grams_from_phrase(phrase: List[Tuple[Tuple[int,int], Set[str]]]):
    result = []
#     for borders, words in phrase:
#         result.extend([(borders[0], (word, )) for word in words])
    if len(phrase) == 1:
        return [(phrase[0][0][0], value) for value in phrase[0][1]]
    
    for n in range(1, min(MAX_N, len(phrase)+1)):
        for i in range(0, len(phrase)-n+1):
            res = ['_'.join(value) for value in itertools.product(*[pair[1] for pair in phrase][i:i+n])]
            res = [v for v in res if v in ALL_FORMS]
            if res:
                result.extend([(phrase[i][0][0], r) for r in res])
    
    return result


def get_n_grams_from_sequence(sequence: List[List[Set[str]]]):
    result = []
    for phrase in sequence:
        result.extend(get_n_grams_from_phrase(phrase))
    return result

In [13]:
# print(normalize_lines[0][2])
# print(get_n_grams_from_phrase(normalize_lines[0][2]))

In [14]:
normalize_lines_with_grams = [get_n_grams_from_sequence(line) for line in normalize_lines]

In [15]:
normalize_lines_with_grams[0]

[(0, 'условие'),
 (8, 'термодинамический'),
 (26, 'согласованность'),
 (42, 'иметь'),
 (48, 'большой'),
 (56, 'значение'),
 (8, 'термодинамический_согласованность'),
 (42, 'иметь_большой'),
 (48, 'большой_значение'),
 (66, 'например'),
 (75, 'при'),
 (79, 'использование'),
 (93, 'уравнение'),
 (103, 'состояние'),
 (113, 'век'),
 (115, 'газодинамический'),
 (132, 'расчёт'),
 (75, 'при_использование'),
 (79, 'использование_уравнение'),
 (93, 'уравнение_состояние'),
 (103, 'состояние_в'),
 (115, 'газодинамический_расчёт'),
 (75, 'при_использование_уравнение'),
 (141, 'век'),
 (143, 'дать'),
 (143, 'данный'),
 (150, 'работа'),
 (157, 'рассматриваться'),
 (173, 'вычисление'),
 (184, 'давление'),
 (193, 'век'),
 (195, 'модель'),
 (202, 'ограниченный'),
 (202, 'ограничить'),
 (216, 'атом'),
 (224, 'показываться'),
 (141, 'в_дать'),
 (141, 'в_данный'),
 (143, 'дать_работа'),
 (150, 'работа_рассматриваться'),
 (184, 'давление_в'),
 (193, 'в_модель'),
 (195, 'модель_ограниченный'),
 (202, 'огран

In [16]:
positions = []
for normalize_line_with_grams in normalize_lines_with_grams:
    dict_positions = {}
    for position, phrase in normalize_line_with_grams:
        dict_positions.setdefault(phrase, []).append(position)
    positions.append(dict_positions)

In [17]:
positions[0]

{'условие': [0],
 'термодинамический': [8],
 'согласованность': [26],
 'иметь': [42],
 'большой': [48],
 'значение': [56],
 'термодинамический_согласованность': [8],
 'иметь_большой': [42],
 'большой_значение': [48],
 'например': [66],
 'при': [75, 242],
 'использование': [79, 246],
 'уравнение': [93],
 'состояние': [103],
 'век': [113, 141, 193, 398, 462, 560],
 'газодинамический': [115],
 'расчёт': [132],
 'при_использование': [75, 242],
 'использование_уравнение': [79],
 'уравнение_состояние': [93],
 'состояние_в': [103],
 'газодинамический_расчёт': [115],
 'при_использование_уравнение': [75],
 'дать': [143],
 'данный': [143],
 'работа': [150, 294],
 'рассматриваться': [157, 509],
 'вычисление': [173],
 'давление': [184],
 'модель': [195, 400, 428, 476],
 'ограниченный': [202, 407, 435],
 'ограничить': [202, 407, 435],
 'атом': [216, 449],
 'показываться': [224],
 'в_дать': [141],
 'в_данный': [141],
 'дать_работа': [143],
 'работа_рассматриваться': [150],
 'давление_в': [184],
 'в_

In [18]:
# normalize_lines_with_grams[0]

In [19]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()
text_vectors = vectorizer.fit(ALL_FORMS)

text_vectors = vectorizer.transform([' '.join([l[1] for l in line]) for line in normalize_lines_with_grams])

In [21]:
reques = 'полезное ископаемое'

request_words = [pair[1] for pair in get_n_grams_from_sequence(normalize_line(reques))]
print('request_words', request_words)

weights = {(1, phrase) for phrase in request_words}

for phrase in request_words:
    weights |= top_down_algorithm(phrase, voc)

print('weights', weights)

vector_reques = sum([weight * vectorizer.transform([phrase]) for weight, phrase in weights])

print('vector_reques\n', vector_reques)

request_words ['полезный', 'ископаемое', 'полезный_ископаемое']
weights {(0.9, 'ресурс'), (0.5, 'нефть'), (1, 'полезный'), (1, 'полезный_ископаемое'), (0.5, 'газ'), (1, 'ископаемое')}
vector_reques
   (0, 7418)	0.5
  (0, 15757)	1.0
  (0, 25078)	0.5
  (0, 31782)	1.0
  (0, 31785)	1.0
  (0, 38269)	0.9


In [22]:
# print(request_words)
# vector_reques = vectorizer.transform([' '.join(request_words)])
# print()
# print(vector_reques)

In [23]:
# print(len(vector_reques.toarray()))
# print(len(vector_reques.toarray()[0]))
# print(vector_reques.shape)
# vector_reques

In [24]:
# print(len(text_vectors.toarray()))
# print(len(text_vectors.toarray()[0]))
# print(text_vectors.shape)
# text_vectors

In [25]:
result = text_vectors*vector_reques.transpose()

In [67]:
# for i,(is_search, line) in enumerate(zip([res[0] for res in result.toarray()], text_lines)):
#     if is_search:
#         print(i)
#         print(is_search)
#         print(line)
#         print()

In [27]:
results_idex = [(i, is_search) for i, (is_search, line) in enumerate(zip([res[0] for res in result.toarray()], text_lines)) if is_search]
results_idex.sort(key=lambda x: x[1], reverse=True)
len(results_idex)

588

In [29]:
i = 0

print(results_idex[i])
line_positions = positions[results_idex[i][0]]
res = {phrase: line_positions[phrase] for _, phrase in weights if phrase in line_positions}
print(res)
result_line = text_lines[results_idex[i][0]]
result_line

(3228, 9.0)
{'полезный': [84, 219, 520], 'полезный_ископаемое': [84, 219, 520], 'ископаемое': [93, 228, 529]}


'Михеева в своем докладе выделяет 21 сырьевой регион РФ, ориентируясь на долю добычи полезных ископаемых в структуре ВРП. Используются также методы, основанные на соотношении объемов валовой добавленной стоимости добычи полезных ископаемых и обрабатывающих производств в субъектах РФ [21]. Для анализа стратегических документов социально-экономического развития сырьевых субъектов РФ в качестве ключевого критерия выделения сырьевых регионов России мы использовали показатель доли валовой добавленной стоимости от добычи полезных ископаемых в структуре ВРП (более 30% в период 2000–2010 гг.'

In [ ]:
result_line

In [30]:
# print_values = vector_reques.toarray()[0]

# for i, v in enumerate(print_values):
#     if v != 0:
#         print(i, v, vectorizer.get_feature_names()[i])